In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf

In [ ]:
img_width = 224
img_height = 224
batch_size = 256
num_of_classes = 70
IMG_SIZE = 224

training_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/train"
validation_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/val"
test_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/test"

train_data = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
)

validation_data = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
)

test_data = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

training_gen = train_data.flow_from_directory(
    training_set_directory,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    target_size=(img_height, img_width)
)

validation_gen = validation_data.flow_from_directory(
    validation_set_directory,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    target_size=(img_height, img_width)
)

test_gen = test_data.flow_from_directory(
    test_set_directory,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    target_size=(img_height, img_width)
)


Found 6542 images belonging to 70 classes.
Found 1857 images belonging to 70 classes.
Found 1000 images belonging to 70 classes.


In [ ]:



base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.6)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.6)(x)
predictions = Dense(num_of_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


for layer in model.layers[-10:]:
    layer.trainable = True


optimizer = Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
checkpoint_filepath = '/tmp/checkpoint2'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [ ]:
total_training_samples = len(training_gen.filenames)
steps_per_epoch = total_training_samples // batch_size

In [ ]:
epochs = 50

history = model.fit(
    training_gen,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_gen,
    verbose=1,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/50
 7/25 [=======>......................] - ETA: 1:00:33 - loss: 29.7491 - accuracy: 0.0179

In [ ]:

model.load_weights(checkpoint_filepath)



In [ ]:

model.save('/tmp/goodcheckpoint.h5')



In [ ]:
model.evaluate(test_gen)


4/4 [==============================] - 4s 925ms/step - loss: 3.1415 - accuracy: 0.7220


[3.1414966583251953, 0.722000002861023]

In [ ]:
optimizer = Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 20

history = model.fit(
    training_gen,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_gen,
    verbose=1,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/20
25/25 [==============================] - 84s 3s/step - loss: 2.8946 - accuracy: 0.8271 - val_loss: 3.2146 - val_accuracy: 0.7518
Epoch 2/20
25/25 [==============================] - 83s 3s/step - loss: 2.8663 - accuracy: 0.8331 - val_loss: 3.2062 - val_accuracy: 0.7458
Epoch 3/20
25/25 [==============================] - 82s 3s/step - loss: 2.8684 - accuracy: 0.8263 - val_loss: 3.1984 - val_accuracy: 0.7485
Epoch 4/20
25/25 [==============================] - 83s 3s/step - loss: 2.8518 - accuracy: 0.8303 - val_loss: 3.1927 - val_accuracy: 0.7550
Epoch 5/20
25/25 [==============================] - 83s 3s/step - loss: 2.8445 - accuracy: 0.8333 - val_loss: 3.1845 - val_accuracy: 0.7561
Epoch 6/20
25/25 [==============================] - 83s 3s/step - loss: 2.8188 - accuracy: 0.8392 - val_loss: 3.1778 - val_accuracy: 0.7518
Epoch 7/20
25/25 [==============================] - 83s 3s/step - loss: 2.8378 - accuracy: 0.8288 - val_loss: 3.1684 - val_accuracy: 0.7523
Epoch 8/20
25/25 [==

In [ ]:
for layer in model.layers[-15:]:
    layer.trainable = True

In [ ]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


fine_tune_epochs = 10
history = model.fit(
    training_gen,
    epochs=epochs + fine_tune_epochs,
    initial_epoch=epochs,
    validation_data=validation_gen,
    verbose=1,
    callbacks=[model_checkpoint_callback]
)

Epoch 101/110
26/26 [==============================] - 97s 3s/step - loss: 1.5944 - accuracy: 0.7661 - val_loss: 1.9372 - val_accuracy: 0.7065
Epoch 102/110
26/26 [==============================] - 91s 4s/step - loss: 1.3484 - accuracy: 0.8341 - val_loss: 1.8749 - val_accuracy: 0.7286
Epoch 103/110
26/26 [==============================] - 90s 4s/step - loss: 1.2635 - accuracy: 0.8510 - val_loss: 1.8572 - val_accuracy: 0.7297
Epoch 104/110
26/26 [==============================] - 90s 3s/step - loss: 1.1936 - accuracy: 0.8751 - val_loss: 1.8447 - val_accuracy: 0.7356
Epoch 105/110
26/26 [==============================] - 90s 3s/step - loss: 1.1573 - accuracy: 0.8786 - val_loss: 1.8232 - val_accuracy: 0.7367
Epoch 106/110
26/26 [==============================] - 93s 4s/step - loss: 1.1213 - accuracy: 0.8912 - val_loss: 1.8871 - val_accuracy: 0.7189
Epoch 107/110
26/26 [==============================] - 91s 4s/step - loss: 1.1140 - accuracy: 0.8893 - val_loss: 1.8124 - val_accuracy: 0.7410